<a href="https://colab.research.google.com/github/luiseduaardo/PneuModel/blob/main/notebooks/inferencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/luiseduaardo/PneuModel.git
%cd PneuModel

Cloning into 'PneuModel'...
remote: Enumerating objects: 5949, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 5949 (delta 20), reused 9 (delta 3), pack-reused 5901 (from 3)
Receiving objects: 100% (5949/5949), 1.26 GiB | 34.58 MiB/s, done.
Resolving deltas: 100% (38/38), done.
/content/PneuModel


In [2]:
!pip install -r requirements.txt

In [3]:
import os
import zipfile

FILE_ID = '1YLeauT4Kwl6EpdpcUesU6WdE5cA12Y4K'
ZIP_NAME = 'ligia-compviz.zip'
EXTRACT_PATH = './dataset'

def download_and_extract():
    if os.path.exists(ZIP_NAME):
        os.remove(ZIP_NAME)

    command = f"""
      FILEID="{FILE_ID}";
      FILENAME="{ZIP_NAME}";
      wget --quiet --save-cookies cookies.txt --keep-session-cookies \
      "https://drive.google.com/uc?export=download&id=$FILEID" -O response.html;
      UUID=$(grep -o 'name="uuid" value="[^"]*' response.html | sed 's/.*value="//');
      wget --load-cookies cookies.txt \
      "https://drive.usercontent.google.com/download?id=$FILEID&export=download&confirm=t&uuid=$UUID" \
      -O "$FILENAME";
      rm -f cookies.txt response.html;
    """

    os.system(command)

    if os.path.exists(ZIP_NAME) and zipfile.is_zipfile(ZIP_NAME):
        print("Arquivo baixado com sucesso. Extraindo...")
        with zipfile.ZipFile(ZIP_NAME, 'r') as zip_ref:
            zip_ref.extractall(EXTRACT_PATH)
        print(f"✅ Dataset pronto em {EXTRACT_PATH}")

download_and_extract()

Arquivo baixado com sucesso. Extraindo...
✅ Dataset pronto em ./dataset


In [4]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import os

In [5]:
drive.mount('/content/drive')

path_teste = "./dataset/test_images"
model_path_drive = '/content/drive/MyDrive/PneuModel/modelo.keras'

model = load_model(model_path_drive)
print("✅ Modelo carregado com sucesso!")

Mounted at /content/drive
✅ Modelo carregado com sucesso!


In [6]:
# pré processamento da imagem
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    path_teste,
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# realiza a previsão
previsoes = model.predict(test_generator)

Found 624 images belonging to 1 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


624/624 ━━━━━━━━━━━━━━━━━━━━ 143s 220ms/step


In [7]:
ids = [os.path.basename(x) for x in test_generator.filenames]

submissao = pd.DataFrame({
    'id': ids,
    'target': previsoes.flatten()
})

submissao.head()

,id,target
0,img_0001.jpeg,0.310470
1,img_0002.jpeg,0.535725
2,img_0003.jpeg,0.998388
3,img_0004.jpeg,0.006515
4,img_0005.jpeg,0.018332


In [8]:
path_final = './submission.csv'
submissao.to_csv(path_final, index=False, sep=',')